<a href="https://colab.research.google.com/github/tomanizer/stats_in_10_minutes/blob/master/NumericalMethods_of_ODEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerical methods for solving ODEs

- Euler's method
- Midpoint method
- Runge-Kutta method

## Euler's method

ODE with initial value problem
$\frac{dx}{dt}=f(x,t) \quad x(t_0)=x_0$

### Problem statement
How do we find estimates of $[x_0, x_1, x_2..]$ at times $[t_0, t_1, ...]$ that are reasonable estimates of $[x(t_0), x(t_1), ...]$ where $x(t_0)$ is the exact solution of the ODE.

Suppose we knot $x(t_k)$ and we want to estimate $x(t_{k+1})$ where $t_{k+1} = t_k + h$ and h is small.

Assume $x(t)$ is locally smooth enough in the vicinity of $t=t_k$ for a Taylor series expansion.

$x(t_{k+1}) = x(t_k + h) = x(t_k) + h \frac{dx}{dt} + \frac{1}{2}h^2 \frac{d^2 x}{dt^2}+ ...$ 

### Truncation

Euler's method is to 
- truncate the Taylor series after *two* terms.
- use the derivative function evalueated at the initial point to take one small step forward

$x_{k+1} = x_k + hf(x_k, t_k)$

### Algorithm

- Divide the domain $[t_0, t_f]$ into n equally-spaced intervals.
- Start with the initial condition $x_0$ at $t_0$
- Iterate

### Example

Solve the intiial value problem $\frac{dx}{dy}=x, \quad x(0) =1$

a) using separation of variables

$\frac{dx}{dy}=x$







## Midpoint Method

In [0]:
import numpy as np
import plotly.graph_objects as go

In [0]:
def midpoint(tmin=0.0, tmax=20.0, n=100):
  """The midpoint method applied to the SHM equation.
  dx/dt = y, dy/dt = -x"""
  ts = np.linspace(tmin, tmax, n+1)
  h = (tmax - tmin)/n
  xs = np.zeros(n+1); ys = np.zeros(n+1)
  xs[0] = 1.0; ys[0] = 0.0
  for k in range(n):
    x1 = xs[k] + 0.5*h*ys[k] # midpoint estimate
    y1 = ys[k] - 0.5*h*xs[k]
    xs[k+1] = xs[k] + h*y1
    ys[k+1] = ys[k] - h*x1
  return ts, xs, ys


## Runge-Kutta

Numerically solve the ODE

$\frac{dx}{dt} = e^{−t/10} sin(t) sin(x), \quad x(0) = 1$

in the range $t ∈ [0, 20]$ using a 3rd order Runge-Kutta
method.

In [0]:
def dx_dt(x, t):
  return np.sin(t)*np.sin(x)*np.exp(-0.1*t)

In [0]:
def rk3_step(x, t, h):
  """See Butcher tableau for RK3 method"""
  k1 = h*dx_dt(x, t)
  k2 = h*dx_dt(x+k1/2.0, t+h/2.0)
  k3 = h*dx_dt(x-k1+2*k2, t+h)
  x_next = x + (k1 + 4*k2 + k3)/6.0
  return x_next

In [0]:
def rk3_solve(x0, h, tmin, tmax):
  n = int((tmax - tmin)/h) # number of steps
  ts = np.zeros(n+1); xs = np.zeros(n+1) # arrays
  xs[0] = x0; ts[0] = tmin # set initial conditions
  for k in range(n):
    ts[k+1] = ts[k] + h
    xs[k+1] = rk3_step(xs[k], ts[k], h)
  return ts, xs

In [0]:
x0 = 1.0; h = 0.01; tmin = 0.0; tmax = 20.0;
ts, xs = rk3_solve(x0, h, tmin, tmax)

In [16]:
fig = go.Figure(layout={"title":"ODE"})
fig.add_scatter(x=ts, y=xs)

# Linear Multi-Step

In [0]:
def twostep_solve(x0, h, tmin, tmax):
  n = int((tmax - tmin)/h) + 1
  ts = np.zeros(n)
  ts[0] = tmin
  xs = np.zeros(n)
  xs[0] = x0 # Use midpoint method for first step.
  xhalf = xs[0] + 0.5*h*dx_dt(xs[0], ts[0])
  xs[1] = xs[0] + h*dx_dt(xhalf, ts[0]+h/2.0)
  # Now apply the two-step method.
  for k in range(1,n-1):
    ts[k+1] = ts[k] + h
    xs[k+1] = xs[k] + (h/2.0)*(3*dx_dt(xs[k], ts[k])
    - dx_dt(xs[k-1], ts[k-1]))
  return ts, xs